<a href="https://colab.research.google.com/github/Meenusj/Case_study/blob/main/reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [6]:
import pandas as pd


df = pd.read_csv('/content/reddit_filtered_dataset.csv')
df.head()


,Data,Labels
0,"Of course! The relevant passage is: ""As to you...",0
1,I've actually written a bit about this before....,0
2,The Waco standoff in 1993 was a complex and co...,1
3,Incredible answer. This was very well sourced ...,0
4,The identification of the ruins of Troy is bas...,1


In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Load MiniLM model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Step 1: Read and Prepare Data

X = df['Data'].values
y = df['Labels'].values

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Splitting data into train, validation, test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 2: Get MiniLM Embeddings
def get_minilm_embeddings(model, texts):
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings.cpu().numpy()  # Convert to NumPy array and move to CPU

X_train_embeddings = get_minilm_embeddings(model, X_train)
X_val_embeddings = get_minilm_embeddings(model, X_val)
X_test_embeddings = get_minilm_embeddings(model, X_test)

# Step 3: Define and compile your model with MiniLM Embeddings
input_shape = X_train_embeddings.shape[1:]

model_cnn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Step 4: Training
history = model_cnn.fit(X_train_embeddings, y_train,
                        epochs=10,
                        batch_size=64,
                        validation_data=(X_val_embeddings, y_val),
                        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

# Step 5: Evaluation
loss, accuracy = model_cnn.evaluate(X_test_embeddings, y_test)
print(f'Test accuracy: {accuracy}')

# Confusion Matrix
y_pred = np.argmax(model_cnn.predict(X_test_embeddings), axis=1)
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)


# Print Actual vs Predicted Outputs
actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print("Actual vs Predicted Outputs:")
print(actual_vs_predicted.head(30))  # Display the first 30 predictions

# Save the model
model_cnn.save('text_classification_model_with_minilm_cnn.h5')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch 1/10
72/72 [==============================] - 4s 6ms/step - loss: 0.5344 - accuracy: 0.7752 - val_loss: 0.4408 - val_accuracy: 0.7871
Epoch 2/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3948 - accuracy: 0.8024 - val_loss: 0.3471 - val_accuracy: 0.8403
Epoch 3/10
72/72 [==============================] - 0s 4ms/step - loss: 0.3160 - accuracy: 0.8592 - val_loss: 0.2808 - val_accuracy: 0.8792
Epoch 4/10
72/72 [==============================] - 0s 4ms/step - loss: 0.2651 - accuracy: 0.8844 - val_loss: 0.2531 - val_accuracy: 0.9017
Epoch 5/10
72/72 [==============================] - 0s 4ms/step - loss: 0.2305 - accuracy: 0.9052 - val_loss: 0.2455 - val_accuracy: 0.9028
Epoch 6/10
72/72 [==============================] - 0s 4ms/step - loss: 0.2102 - accuracy: 0.9118 - val_loss: 0.2429 - val_accuracy: 0.9038
Epoch 7/10
72/72 [==============================] - 0s 4ms/step - loss: 0.2010 - accuracy: 0.9162 - val_loss: 0.2309 - val_accuracy: 0.9069
Epoch 8/10
72/72 [==

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Load MiniLM model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Step 1: Read and Prepare Data
# Assuming df is already loaded and contains 'Data' and 'Labels' columns
X = df['Data'].values
y = df['Labels'].values

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Splitting data into train, validation, test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 2: Get MiniLM Embeddings
def get_minilm_embeddings(model, texts):
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings.cpu().numpy()  # Convert to NumPy array and move to CPU

X_train_embeddings = get_minilm_embeddings(model, X_train)
X_val_embeddings = get_minilm_embeddings(model, X_val)
X_test_embeddings = get_minilm_embeddings(model, X_test)

# Step 3: Define and compile your model with MiniLM Embeddings
input_shape = X_train_embeddings.shape[1:]

model_cnn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_cnn.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Step 4: Training
history = model_cnn.fit(X_train_embeddings, y_train,
                        epochs=15,
                        batch_size=32,
                        validation_data=(X_val_embeddings, y_val),
                        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

# Step 5: Evaluation
loss, accuracy = model_cnn.evaluate(X_test_embeddings, y_test)
print(f'Test accuracy: {accuracy}')

# Confusion Matrix
y_pred = np.argmax(model_cnn.predict(X_test_embeddings), axis=1)
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

# Print Actual vs Predicted Outputs
actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print("Actual vs Predicted Outputs:")
print(actual_vs_predicted.head(30))  # Display the first 30 predictions

# Save the model
model_cnn.save('text_classification_model_with_minilm_cnn.h5')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch 1/15
143/143 [==============================] - 2s 5ms/step - loss: 0.4457 - accuracy: 0.7923 - val_loss: 0.3327 - val_accuracy: 0.8536
Epoch 2/15
143/143 [==============================] - 1s 6ms/step - loss: 0.2810 - accuracy: 0.8826 - val_loss: 0.2491 - val_accuracy: 0.8987
Epoch 3/15
143/143 [==============================] - 1s 6ms/step - loss: 0.2136 - accuracy: 0.9118 - val_loss: 0.2314 - val_accuracy: 0.9048
Epoch 4/15
143/143 [==============================] - 1s 5ms/step - loss: 0.1639 - accuracy: 0.9329 - val_loss: 0.2422 - val_accuracy: 0.9048
Epoch 5/15
143/143 [==============================] - 1s 4ms/step - loss: 0.1218 - accuracy: 0.9542 - val_loss: 0.2588 - val_accuracy: 0.9069
Epoch 6/15
31/31 [==============================] - 0s 2ms/step - loss: 0.2393 - accuracy: 0.8987
Test accuracy: 0.898669421672821
31/31 [==============================] - 0s 1ms/step
Confusion Matrix:
[[736  44]
 [ 55 142]]
Actual vs Predicted Outputs:
    Actual  Predicted
0        1    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
import pandas as pd

# Load the CSV file
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path, sep=';')

# Convert 'account.type' to 0 and 1
df['account.type'] = df['account.type'].map({'human': 0, 'bot': 1})

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Load MiniLM model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Step 1: Read and Prepare Data
# Assuming df is already loaded and contains 'Data' and 'Labels' columns
X = df['text'].values
y = df['account.type'].values

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Splitting data into train, validation, test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 2: Get MiniLM Embeddings
def get_minilm_embeddings(model, texts):
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings.cpu().numpy()  # Convert to NumPy array and move to CPU

X_train_embeddings = get_minilm_embeddings(model, X_train)
X_val_embeddings = get_minilm_embeddings(model, X_val)
X_test_embeddings = get_minilm_embeddings(model, X_test)

# Step 3: Define and compile your model with MiniLM Embeddings
input_shape = X_train_embeddings.shape[1:]

model_cnn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_cnn.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Step 4: Training
history = model_cnn.fit(X_train_embeddings, y_train,
                        epochs=15,
                        batch_size=32,
                        validation_data=(X_val_embeddings, y_val),
                        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

# Step 5: Evaluation
loss, accuracy = model_cnn.evaluate(X_test_embeddings, y_test)
print(f'Test accuracy: {accuracy}')

# Confusion Matrix
y_pred = np.argmax(model_cnn.predict(X_test_embeddings), axis=1)
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

# Print Actual vs Predicted Outputs
actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print("Actual vs Predicted Outputs:")
print(actual_vs_predicted.head(30))  # Display the first 30 predictions

# Save the model
model_cnn.save('text_classification_model_with_minilm_cnn.h5')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch 1/15
454/454 [==============================] - 3s 4ms/step - loss: 0.4993 - accuracy: 0.7500 - val_loss: 0.4097 - val_accuracy: 0.8027
Epoch 2/15
454/454 [==============================] - 2s 4ms/step - loss: 0.4018 - accuracy: 0.8092 - val_loss: 0.3842 - val_accuracy: 0.8194
Epoch 3/15
454/454 [==============================] - 2s 5ms/step - loss: 0.3587 - accuracy: 0.8296 - val_loss: 0.3720 - val_accuracy: 0.8214
Epoch 4/15
454/454 [==============================] - 6s 12ms/step - loss: 0.3253 - accuracy: 0.8509 - val_loss: 0.3683 - val_accuracy: 0.8191
Epoch 5/15
454/454 [==============================] - 3s 6ms/step - loss: 0.2930 - accuracy: 0.8659 - val_loss: 0.3733 - val_accuracy: 0.8230
Epoch 6/15
454/454 [==============================] - 2s 4ms/step - loss: 0.2618 - accuracy: 0.8820 - val_loss: 0.3827 - val_accuracy: 0.8256
Epoch 7/15
98/98 [==============================] - 0s 2ms/step - loss: 0.3710 - accuracy: 0.8304
Test accuracy: 0.8303830027580261
98/98 [========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
